In [1]:
from datasets import load_dataset
import json

target_ds = load_dataset("singletongue/wikipedia-utils", "passages-c400-jawiki-20230403", split="train")

jsonl_path = 'wikipedia_passages_c400_jawiki_20230403.jsonl'
with open(jsonl_path, 'w', encoding='utf-8') as f:
    count = 0
    for item in target_ds:
        if count < 1025:
            json.dump(item, f, ensure_ascii=False)
            f.write('\n')
            count += 1
        else:
            break

/home/ubuntu/.cache/pypoetry/virtualenvs/llm-rag-kit-2uEd5D3k-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from datasets import load_dataset

target_ds = load_dataset('json', data_files="wikipedia_passages_c400_jawiki_20230403.jsonl", split="train")

for item in target_ds:
    print(item)

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1501.18it/s]
Generating train split: 150 examples [00:00, 38479.85 examples/s]


In [1]:
from sentence_transformers import SentenceTransformer
from hf_hub_ctranslate2 import CT2SentenceTransformer


MODEL = CT2SentenceTransformer("/home/ubuntu/llm-rag-kit/multilingual-e5-large", compute_type="float16")

/home/ubuntu/.cache/pypoetry/virtualenvs/llm-rag-kit-2uEd5D3k-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets.download import DownloadManager
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import faiss

In [4]:
# データセットのロード
wikija_dataset = load_dataset('json', data_files="wikipedia_passages_c400_jawiki_20230403.jsonl", split="train")
index_local_path = "./outputs/faiss_indexes/multilingual-e5-large-passage/index_IVF256_PQ64.faiss"
# faiss index のロード
faiss_index = faiss.read_index(index_local_path)

In [6]:
model = SentenceTransformer("/home/ubuntu/llm-rag-kit/multilingual-e5-large")
model.max_seq_length = 512

In [7]:
def to_emb(model, text, prefix="query: "):
    return model.encode([prefix + text], normalize_embeddings=True)


emb = to_emb(model, "Apex Legendsで問題になった出来事は?")

In [8]:
TOP_K = 5
scores, indexes = faiss_index.search(emb, TOP_K)
for idx, (id, score) in enumerate(zip(indexes[0], scores[0])):
    data = wikija_dataset[int(id)]
    print((score, data["title"], data["text"][:100]))

(0.45435277, '人工知能', 'Apex Legendsなどの人気ゲームに出演する声優のフリーダ・ウォルフは合意や追加の補償なしに自身の音声データを利用する搾取に繋がるとして懸念を表明したほか、フォートナイトなどの声優であるサラ・エ')
(0.4594062, '音楽', '日本では、縄文時代から既に音楽が始まっていたが、5世紀から8世紀にかけて朝鮮半島・中国から音楽を取り入れたことからさまざまなジャンルの音楽が始まった。まず、平安時代に遣唐使を廃止して国風文化が栄えてい')
(0.46018404, '日本語', '例えば以下の文体は、実際の性別・博士・令嬢・地方出身者などの一般的な位相を反映したものではないものの、小説・漫画・アニメ・ドラマなどで、仮想的にそれらしい感じを与える文体として広く観察される。これは現')
(0.46510434, '日本語', 'そこで、文部省は新しい語法のうち一部慣用の久しいものを認め、「文法上許容スベキ事項」(1905年・明治38年)16条を告示した。一方、明治20年代頃から、二葉亭四迷・山田美妙ら文学者を中心に、書き言葉')
(0.4653516, '日本語', '受け身の表現において、人物以外が主語になる例は、近代以前には乏しい。もともと、日本語の受け身表現は、自分の意志ではどうにもならない「自然生起」の用法の一種であった。したがって、物が受け身表現の主語にな')
